In [205]:
import xml.etree.ElementTree as etree
from lxml.etree import iterparse
from multiprocessing import Process, JoinableQueue, Pool
from tqdm import tqdm
from corenlp_pywrap import pywrap
import re

In [133]:
def iterate_wiki_xml(queue):
    count = 0
    for event, elem in etree.iterparse('datasets/wikipedia/wiki.xml', events=['start', 'end']):
        if elem.tag == ns + 'page' and event == 'end': 
            print("added page to queue")
            queue.put(elem)
            count += 1
        if count > 1:
            break
        print("iterating")
#         if elem.tag == ns + 'page' and event == 'start':
#             elem.clear()

In [30]:
ns = "{http://www.mediawiki.org/xml/export-0.10/}"
lines = 945682732

In [167]:
f = open('datasets/wikipedia/wiki.xml', 'rb')
indices = []
tqdm().close()
index = 0
for line in tqdm(f, total=lines, position=0):
    line = line.decode('utf-8')
    if "<page" in line:
        indices.append(index)
    index = f.tell()
indices
f.close()


0it [00:00, ?it/s]
945682733it [22:17, 706981.93it/s]                               


In [243]:
import _pickle
with open('datasets/wikipedia/indices.pkl', 'wb') as f:
    _pickle.dump(indices, f)

In [217]:
_MEDIA_CAT = """
  [Ii]mage|[Cc]ategory      # English
 |[Aa]rchivo                # Spanish
 |[Ff]ile                   # English, Italian
 |[CcKk]at[ée]gor[íi][ea]   # Dutch, German, French, Italian, Spanish, Polish, Latin
 |[Bb]estand                # Dutch
 |[Bb]ild                   # German
 |[Ff]icher                 # French
 |[Pp]lik                   # Polish
 |[Ff]asciculus             # Latin
"""

_UNWANTED = re.compile(r"""
  (:?
    \{\{ .*? \}\}                           # templates
  | \| .*? \n                               # left behind from templates
  | \}\}                                    # left behind from templates
  | <!-- .*? -->
  | <div .*?> .*? </div>
  | <math> .*? </math>
  | <nowiki> .*? </nowiki>
  | <ref .*?> .*? </ref>
  | <ref .*?/>
  | <span .*?> .*? </span>
  | \[\[ (:?%s): (\[\[.*?\]\]|.)*? \]\]
  | \[\[ [a-z]{2,}:.*? \]\]                 # interwiki links
  | =+                                      # headers
  | \{\| .*? \|\}
  | \[\[ (:? [^]]+ \|)?
  | \]\]
  | '{2,}
  )
""" % _MEDIA_CAT,
re.DOTALL | re.MULTILINE | re.VERBOSE)


def text_only(text):
    return _UNWANTED.sub("", text)

In [208]:
def process(index, nextIndex):
    with open('datasets/wikipedia/wiki.xml', 'rb') as f:
        f.seek(index)
        data = f.read(nextIndex - index).decode('utf-8')
        elem = etree.fromstring(data)
        print(elem)
        doc = process_page(elem)
    if doc:
        c['corpora']['wiki'].insert_one(doc)
    return doc

In [242]:
list(zip(indices, indices[1:] + [None]))[0]

(2848, 3512)

In [46]:
elem.findall(".//{}revision/{}text".format(ns, ns))[0].text
# elem.findall(".//{}title".format(ns))[0].text

'#REDIRECT [[Computer accessibility]]\n\n{{Redr|move|from CamelCase|up}}'

In [238]:
def process_page(elem):
    full_annotator_list = ["ssplit"]
    cn = pywrap.CoreNLP(url='http://172.17.0.2:9000', annotator_list = full_annotator_list)
    text = elem.findall(".//revision/text")[0].text
    title = elem.findall(".//title")[0].text
    text = text_only(text)
    if text.startswith("#REDIRECT"):
        return
    paras = re.split(r'\n', text)
    doc = {}
    doc['title'] = title
    doc['paras'] = []
    for para in paras:
        if para:
            para = para.encode('latin-1', 'ignore').decode('latin-1')
            sentences = cn.basic(para).json()['sentences']
            endOfLast = 0
            doc['paras'].append([])
            for sent in sentences:
                last_word = sent['tokens'][-1]
                sentence_text = para[endOfLast:last_word['characterOffsetEnd']]
                endOfLast = last_word['characterOffsetEnd']
                tokens = list(map(lambda x: x['word'], sent['tokens']))
                doc['paras'][-1].append({'text': sentence_text, 'tokens': tokens})        
    return doc

In [188]:
process(indices[0])

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 2221: ordinal not in range(128)

In [139]:
queue = JoinableQueue(0)
p = Process(target=iterate_wiki_xml, args = (queue,))
p.daemon = False
p.start()
p.join()
pool = Pool(4, worker_main, (queue,))

print("processed finished")

queue.join()
print("queue finished")
pool.terminate()

iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating
iterating


Process Process-203:
  File "/usr/lib/python3.4/threading.py", line 1060, in join
    self._wait_for_tstate_lock()
Traceback (most recent call last):
  File "/usr/lib/python3.4/multiprocessing/process.py", line 257, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.4/multiprocessing/util.py", line 312, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.4/multiprocessing/util.py", line 252, in _run_finalizers
    finalizer()


KeyboardInterrupt: 

  File "/usr/lib/python3.4/multiprocessing/util.py", line 185, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/usr/lib/python3.4/multiprocessing/queues.py", line 196, in _finalize_join
    thread.join()
  File "/usr/lib/python3.4/threading.py", line 1076, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [ ]:
queue.get(False)

In [101]:
import os
def worker_main(queue):
    while True:
        item = queue.get(True)
        print(os.getpid(), process_page(elem))
        queue.task_done()

In [258]:
index = indices[10000]
nextIndex = indices[10001]
with open('datasets/wikipedia/wiki.xml', 'rb') as f:
    f.seek(index)
    if not nextIndex:
        data = f.read().decode('utf-8')
    else:
       data = f.read(nextIndex - index).decode('utf-8')
    elem = etree.fromstring(data)


In [259]:
text = elem.findall(".//revision/text")[0].text
title = elem.findall(".//title")[0].text


In [300]:
list(filter(lambda x: len(x) > 25, map(text_only, re.split('==.*?==|\n[\n]+', text))))[4]

'Various Christian feast dates were used for the New Year in Europe during the Middle Ages, including March 25 (Feast of the Annunciation) and December 25. However, medieval calendars were still displayed in the Roman fashion with twelve columns from January to December. Beginning in the 16th century, European countries began officially making January 1 the start of the New Year once again—sometimes called Circumcision Style because this was the date of the Feast of the Circumcision, being the seventh day after December 25.'

In [270]:
text

'{{About|the month}}\n{{Redirect|Jan.|other uses|Jan (disambiguation){{!}}Jan}}\n{{pp-move-indef}}\n{{Month header}}\n{{calendar}}\n\'\'\'January\'\'\' is the first [[month]] of the [[year]] in the [[Julian calendar|Julian]] and [[Gregorian calendar]]s and the first month to have the length of 31 days. The first day of the month is known as [[New Year\'s Day]]. It is, on average, the coldest month of the year within most of the [[Northern Hemisphere]] (where it is the second month of [[winter]]) and the warmest month of the year within most of the [[Southern Hemisphere]] (where it is the second month of [[summer]]). In the Southern hemisphere, January is the seasonal equivalent of July in the Northern hemisphere and vice versa.\n\n[[Ancient Roman]] observances during this month include [[Cervula]], and [[Juvenalia]]; celebrated January 1, as well as one of three  [[Agonalia]], celebrated January 9, and [[Carmentalia]], celebrated January 11. These dates do not correspond to the modern 

In [302]:
def process_page(elem):
    full_annotator_list = ["ssplit"]
    cn = pywrap.CoreNLP(url='http://172.17.0.4:9000', annotator_list = full_annotator_list)
    text = elem.findall(".//revision/text")[0].text
    title = elem.findall(".//title")[0].text
    if text.startswith("#REDIRECT") or text.startswith('#redirect'):
        return
    paras = filter(lambda x: len(x) > 25, map(text_only, re.split(r'==.*?==|\n[\n]+', text)))
    doc = {}
    doc['title'] = title
    doc['paras'] = []
    for para in paras:
        para = para.encode('latin-1', 'backslashreplace').decode('latin-1')
        if re.match(r'[\S]+', para):
            sentences = None
            while not sentences:
                try:    
                    sentences = cn.basic(para).json()['sentences']
                except Exception as e:
                    print(e)
                    if e != "Check your CoreNLP Server status \n if not sure, Check the pywrap doc for Server instantiation":
                        print("error \n\n\n\ ", e)
                        return
            endOfLast = 0 
            doc['paras'].append([])
            for sent in sentences:
                last_word = sent['tokens'][-1]
                sentence_text = para[endOfLast:last_word['characterOffsetEnd']]
                endOfLast = last_word['characterOffsetEnd']
                tokens = list(map(lambda x: x['word'], sent['tokens']))
                doc['paras'][-1].append({'text': sentence_text, 'tokens': tokens})        
    if doc['paras']:
        return doc


In [305]:
process_page(elem)['paras'][0]

[{'text': 'Ancient Roman observances during this month include Cervula, and Juvenalia; celebrated January 1, as well as one of three  Agonalia, celebrated January 9, and Carmentalia, celebrated January 11.',
  'tokens': ['Ancient',
   'Roman',
   'observances',
   'during',
   'this',
   'month',
   'include',
   'Cervula',
   ',',
   'and',
   'Juvenalia',
   ';',
   'celebrated',
   'January',
   '1',
   ',',
   'as',
   'well',
   'as',
   'one',
   'of',
   'three',
   'Agonalia',
   ',',
   'celebrated',
   'January',
   '9',
   ',',
   'and',
   'Carmentalia',
   ',',
   'celebrated',
   'January',
   '11',
   '.']},
 {'text': ' These dates do not correspond to the modern Gregorian calendar.',
  'tokens': ['These',
   'dates',
   'do',
   'not',
   'correspond',
   'to',
   'the',
   'modern',
   'Gregorian',
   'calendar',
   '.']}]

In [307]:
from time import sleep
sleep(1)

ImportError: No module named 'JSONDecodeError'